In [32]:
import pandas as pd
import datetime as dt
import numpy as np
import graphviz
import warnings 
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid", color_codes=True)
from sklearn import cross_validation 
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import pickle

In [2]:
ifr_data= pd.read_excel("IFR_FINAL1.xlsx",sep='|')

In [3]:
data1=ifr_data[['AU_ID','Branch','prop_gerrit','source_gerrit','total_file_count','Avg_nloc','Avg_CCN','Avg_fcnt','Job_status']]

In [4]:
data2=data1[data1['AU_ID'].notnull()]

In [5]:
data3=data2.copy()

In [6]:
data3['prop_gerrit'] = data3['prop_gerrit'].apply(lambda x: x*100)

In [7]:
data3['source_gerrit']=data3['source_gerrit'].apply(lambda x : x*100)

In [8]:
data3.shape

(381, 9)

In [9]:
data3['Job_status'].value_counts()

Success    360
Fail        21
Name: Job_status, dtype: int64

In [10]:
def phase(i):
    if i=='Pre_cs':
        return 0
    else:
        return 1

In [11]:
features1=['Branch','prop_gerrit','source_gerrit','total_file_count','Avg_nloc','Avg_CCN','Avg_fcnt']

In [12]:

features=['prop_gerrit','source_gerrit','total_file_count','Avg_CCN','Avg_nloc','Avg_fcnt']

In [13]:
data=data3[features].copy()

In [14]:
classes_data=data3['Job_status'].copy()

In [15]:
data_nsc=data.copy()

In [16]:
data_nsc.shape

(381, 6)

## Mean scaling

In [17]:
from sklearn.model_selection import train_test_split
train,test,classes_train,classes_test = train_test_split(data, classes_data,stratify=classes_data, test_size=0.25)

In [18]:
features

['prop_gerrit',
 'source_gerrit',
 'total_file_count',
 'Avg_CCN',
 'Avg_nloc',
 'Avg_fcnt']

In [297]:
scaled_features = {}
for each in features:
    scaled_features[each_mean]=train[each].mean()
    scaled_features[each_std]=train[each].std()
    print(scaled_features[each_mean])
    #print(scaled_features[each])
    #print(features)
    ##train.loc[:, each] = (train[each] - mean)/std
    #test.loc[:, each] = (test[each] - mean)/std

67.6486650003509
32.35130773649122
69.38245614035088
320.10035225828074
2079.2564616070176
48.074834400382436


In [47]:

def mean(train):
    mean={}
    mean['prop_gerrit']=train['prop_gerrit'].mean()
    mean['source_gerrit']=train['source_gerrit'].mean()
    mean['total_file_count']=train['total_file_count'].mean()
    mean['Avg_CCN']=train['Avg_CCN'].mean()
    mean['Avg_nloc']=train['Avg_nloc'].mean()
    mean['Avg_fcnt']=train['Avg_fcnt'].mean()
    return pd.Series(mean,name="Index")
    
    

In [50]:
MEAN=mean(train)

In [51]:
MEAN

Avg_CCN              328.648553
Avg_fcnt              48.190497
Avg_nloc            2098.295270
prop_gerrit           67.310797
source_gerrit         32.689013
total_file_count      68.449123
Name: Index, dtype: float64

In [52]:
with open('mean.pickle', 'wb') as f:  # Python 3: open(..., 'wb')\n",
    pickle.dump([MEAN], f)

In [53]:
f = open('mean.pickle', 'rb')   # 'rb' for reading binary file
m = pickle.load(f)     
f.close() 


In [54]:
print(m)

[Avg_CCN              328.648553
Avg_fcnt              48.190497
Avg_nloc            2098.295270
prop_gerrit           67.310797
source_gerrit         32.689013
total_file_count      68.449123
Name: Index, dtype: float64]


In [61]:
def mean(features):
    scaled_features={}
    for each in features:
        scaled_features[each]=train[each].mean()
    return scaled_features   

In [64]:
mean(features)

{'Avg_CCN': 328.64855277631915,
 'Avg_fcnt': 48.19049661898244,
 'Avg_nloc': 2098.295269972104,
 'prop_gerrit': 67.31079722070176,
 'source_gerrit': 32.689013316140354,
 'total_file_count': 68.44912280701755}

In [63]:
def std(features):
    scaled_features={}
    for each in features:
        scaled_features[each]=train[each].std()
    return scaled_features   

In [65]:
std(features)

{'Avg_CCN': 257.3690347132846,
 'Avg_fcnt': 32.24766786002248,
 'Avg_nloc': 1919.7075865262314,
 'prop_gerrit': 30.425161548083818,
 'source_gerrit': 30.42489463028296,
 'total_file_count': 189.15786789585013}

In [ ]:
scaled_features = {}
for each in features:
    scaled_features[each_mean]=train[each].mean()
    scaled_features[each_std]=train[each].std()
    print(scaled_features[each_mean])
    #print(scaled_features[each])
    #print(features)
    ##train.loc[:, each] = (train[each] - mean)/std
    #test.loc[:, each] = (test[each] - mean)/std

In [108]:
#data["Branch"] = data["Branch"].apply(phase)

In [109]:
data.head(5)

,prop_gerrit,source_gerrit,total_file_count,Avg_CCN,Avg_nloc,Avg_fcnt
0,19.230769,80.769231,193.0,216.829015,1355.466321,34.367876
194,27.647100,72.352900,449.0,186.371900,2790.788000,25.645880
644,18.636400,81.363600,270.0,252.644400,1416.715000,44.040740
915,19.108300,80.891700,591.0,217.191200,1847.670000,34.768190
1507,21.600000,78.400000,566.0,175.445200,1610.300000,30.045940


## Stratified sampling

In [115]:
from sklearn.model_selection import train_test_split
train,test,classes_train,classes_test = train_test_split(data, classes_data,stratify=classes_data, test_size=0.35) 
                                                   

In [89]:
classes_train.value_counts()

Success    233
Fail        14
Name: Job_status, dtype: int64

In [90]:
classes_test.value_counts()

Success    127
Fail         7
Name: Job_status, dtype: int64

## Random sampling

In [42]:
train1,test1,classes_train1,classes_test1 = cross_validation.train_test_split(data,classes_data,test_size=0.3)

In [43]:
#train['Branch'].value_counts()

In [44]:
train1.head(3)

,prop_gerrit,source_gerrit,total_file_count,Avg_CCN,Avg_nloc,Avg_fcnt
27768,100.000000,0.000000,7.0,195.571429,1439.142857,11.000000
16536,91.666667,8.333333,503.0,5.671968,27.842942,0.827038
24581,91.666667,8.333333,14.0,938.142857,7234.785714,129.714286


In [45]:
classes_train1.value_counts()

Success    253
Fail        13
Name: Job_status, dtype: int64

## Cross validation

In [247]:
from sklearn.model_selection import cross_val_score
clf = LinearSVC(C=1.0,random_state=0,class_weight='balanced')
#rf = RandomForestClassifier(n_estimators=10,class_weight='balanced')
scores = cross_val_score(clf, data, classes_data, cv=5)
#scores1 = cross_val_score(rf, data, classes_data, cv=5)

In [248]:
scores

array([ 0.87012987,  0.94736842,  0.94736842,  0.94736842,  0.94736842])

In [249]:
#scores1

In [250]:
def avg(s):
    t=0
    for i in s:
        t+=i
    return (t/len(s))

In [251]:
avg(scores)

0.93192071086807926

In [96]:
#avg(scores1)

## Linear SVC

#### Stratified 

In [258]:
clf_lv = LinearSVC(C=1.0,random_state=0,class_weight='balanced')
clf_lv.fit(train, classes_train)
y_predict_lv=clf_lv.predict(test)
pd.crosstab(classes_test,y_predict_lv,rownames=['actual'],colnames=['predicted'], margins=True)

predicted,Fail,Success,All
actual,,,
Fail,7,4,11
Success,26,154,180
All,33,158,191


In [259]:
clf_lv.score(test,classes_test)

0.84293193717277481

In [260]:
print(classification_report(classes_test, y_predict_lv))

             precision    recall  f1-score   support

       Fail       0.21      0.64      0.32        11
    Success       0.97      0.86      0.91       180

avg / total       0.93      0.84      0.88       191



#### Random

In [100]:
clf_lv1 = LinearSVC(C=1.0,random_state=0,class_weight='balanced')
clf_lv1.fit(train1, classes_train1)
y_predict_lv1=clf_lv1.predict(test1)
pd.crosstab(classes_test1,y_predict_lv1,rownames=['actual'],colnames=['predicted'], margins=True)

predicted,Fail,Success,All
actual,,,
Fail,6,3,9
Success,24,82,106
All,30,85,115


In [102]:
clf_lv1.score(test,classes_test)

0.8125

In [105]:
print(classification_report(classes_test1, y_predict_lv1))

             precision    recall  f1-score   support

       Fail       0.20      0.67      0.31         9
    Success       0.96      0.77      0.86       106

avg / total       0.90      0.77      0.82       115



### Roc curve

In [33]:
from sklearn import metrics

In [55]:
#fpr, tpr, thresholds = metrics.roc_curve(classes_test, y_predict_lv, pos_label=2)

In [148]:
from sklearn.model_selection import train_test_split
train,test,classes_train,classes_test = train_test_split(data_nsc, classes_data,stratify=classes_data, test_size=0.25) 
                                                   

## Random forest Classifier

In [64]:
from sklearn.ensemble import RandomForestClassifier

In [74]:
clf_rf = RandomForestClassifier(n_estimators=100,max_features='auto',class_weight='balanced')
clf_rf = clf_rf.fit(train, classes_train)
clf_rf.score(test, classes_test)

0.94776119402985071

In [75]:
clf_rf

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [76]:
#clf_rf.feature_importances_

In [77]:
#features

In [78]:
y_predict_rf=clf_rf.predict(test)
pd.crosstab(classes_test,y_predict_rf,rownames=['actual'],colnames=['predicted'], margins=True)

predicted,Fail,Success,All
actual,,,
Fail,1,6,7
Success,1,126,127
All,2,132,134


## Gaussian naive bayes

In [177]:
from sklearn.model_selection import train_test_split
train,test,classes_train,classes_test = train_test_split(data_nsc, classes_data,stratify=classes_data, test_size=0.35) 

In [243]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train,classes_train)

GaussianNB(priors=None)

In [244]:
gnb.score(test,classes_test)

0.86086956521739133

In [245]:
y_predict_gnb=gnb.predict(test)

In [246]:
pd.crosstab(classes_test,y_predict_gnb,rownames=['actual'],colnames=['predicted'], margins=True)

predicted,Fail,Success,All
actual,,,
Fail,4,2,6
Success,14,95,109
All,18,97,115
